# Implemente en un punto de conexión en línea

Para consumir un modelo desde una aplicación, puede implementar el modelo en un punto de conexión en línea. Creará un modelo de MLflow a partir de archivos locales y probará el punto de conexión.

## Antes de comenzar

Necesitará la versión más reciente del paquete **azureml-ai-ml** para ejecutar el código en este cuaderno. Ejecute la celda siguiente para comprobar que está instalada.

> **Nota**:
> Si el paquete **azure-ai-ml** no está instalado, ejecute `pip install azure-ai-ml` para instalarlo.

In [ ]:
## Conexión con su área de trabajo

Con los paquetes de SDK necesarios instalados, ya está listo para conectarse al área de trabajo.

Para conectarse a un espacio de trabajo, se necesitan parámetros de identificación: un id. de suscripción, un nombre de grupo de recursos y un nombre de espacio de trabajo. El nombre del grupo de recursos y el nombre del área de trabajo ya se rellenan automáticamente. Solo necesita el identificador de suscripción para completar el comando.

Para buscar los parámetros necesarios, haga clic en la suscripción y el nombre del área de trabajo en la parte superior derecha de Studio. Se abrirá un panel a la derecha.

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> Copie el identificador de suscripción y reemplace **YOUR-SUBSCRIPTION-ID** por el valor que copió. </p>

## Definición y creación de un punto de conexión

En última instancia, el objetivo es implementar un modelo en un punto de conexión. Por lo tanto, primero debe crear un punto de conexión. El punto de conexión será un punto de conexión HTTPS al que una aplicación puede llamar para recibir predicciones del modelo. Una aplicación puede consumir un punto de conexión mediante su URI y autenticarse con una clave o token.

Ejecute la celda siguiente para definir el punto de conexión. Tenga en cuenta que el nombre del punto de conexión debe ser único. Usará la función `datetime` para generar un nombre único.

In [ ]:
A continuación, creará el punto de conexión mediante la ejecución de la celda siguiente. Este proceso podría tardar varios minutos. Mientras se crea su punto de conexión, puede leer sobre [qué son los puntos de conexión de Azure Machine Learning](https://learn.microsoft.com/azure/machine-learning/concept-endpoints).

In [ ]:
<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> IMPORTANTE: Espere hasta que el punto de conexión se cree correctamente antes de continuar. Debería aparecer una notificación verde en Studio. </p>

## Configurar la implementación

Puede implementar varios modelos en un punto de conexión. Esto es especialmente útil cuando desea actualizar el modelo implementado mientras mantiene el modelo actual en producción. Deberá configurar la implementación para especificar qué modelo debe implementarse en un punto de conexión. En la celda siguiente, hará referencia al modelo entrenado y almacenado en la carpeta local `model` (almacenada en la misma carpeta que este cuaderno). Tenga en cuenta que, dado que está trabajando con un modelo de MLflow, no es necesario especificar el entorno ni el script de puntuación.

También especificará la infraestructura necesaria para que se implemente el modelo.

## Creación de la implementación

Por último, puede implementar realmente el modelo en el punto de conexión mediante la ejecución de la celda siguiente:

In [ ]:
La implementación del modelo puede tardar entre 10 y 15 minutos. Mientras espera a que se implemente el modelo, puede obtener más información sobre los [puntos de conexión administrados en este vídeo](https://www.youtube.com/watch?v=SxFGw_OBxNM&ab_channel=MicrosoftDeveloper).

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> IMPORTANTE: Espere hasta que se complete la implementación antes de continuar. Debería aparecer una notificación verde en Studio.</p>

Como solo tiene un modelo implementado en el punto de conexión, quiere que esta implementación se lleve el 100 % del tráfico. Si implementa varios modelos en el punto de conexión, podría usar el mismo enfoque para distribuir el tráfico entre los modelos implementados.

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> IMPORTANTE: Espere hasta que se configure la implementación azul antes de continuar. Debería aparecer una notificación verde en Studio. </p> 

## Prueba de la implementación

Se probará el modelo implementado invocando el punto de conexión. Como entrada se usa un archivo JSON con datos de ejemplo. El modelo entrenado predice si un paciente tiene diabetes o no, en función de los datos médicos, como la edad, el IMC y el número de embarazos. Un `[0]` indica que el paciente no tiene diabetes. Un `[1]` significa que el paciente tiene diabetes.

In [ ]:
Opcionalmente, puede cambiar los valores del archivo `sample-data.json` para probar y obtener una predicción diferente.

## Enumerar puntos de conexión

Aunque puede ver todos los puntos de conexión en Studio, también puede enumerar todos los puntos de conexión mediante el SDK:

In [ ]:
## Obtención de los detalles del punto de conexión

Si quiere más información sobre un punto de conexión específico, también puede explorar los detalles mediante el SDK.

## Eliminación del punto de conexión y la implementación

Como un punto de conexión siempre está disponible, no se puede pausar para ahorrar costos. Para evitar costos innecesarios, elimine el punto de conexión.

In [ ]:
ml_client.online_deployments.begin_create_or_update(blue_deployment).result()

The deployment of the model may take 10-15 minutes. While waiting for the model to be deployed, you can learn more about [managed endpoints in this video](https://www.youtube.com/watch?v=SxFGw_OBxNM&ab_channel=MicrosoftDeveloper).

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> IMPORTANT! Wait until the deployment is completed before continuing! A green notification should appear in the studio.</p>

Since you only have one model deployed to the endpoint, you want this deployment to take 100% of the traffic. If you deploy multiple models to the endpoint, you could use the same approach to distribute traffic across the deployed models.

In [ ]:
# blue deployment takes 100 traffic
endpoint.traffic = {"blue": 100}
ml_client.begin_create_or_update(endpoint).result()

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> IMPORTANT! Wait until the blue deployment is configured before continuing! A green notification should appear in the studio. </p> 

## Test the deployment

Let's test the deployed model by invoking the endpoint. A JSON file with sample data is used as input. The trained model predicts whether a patient has diabetes or not, based on medical data like age, BMI, and the number of pregnancies. A `[0]` indicates a patient doesn't have diabetes. A `[1]` means a patient does have diabetes.

In [ ]:
# test the blue deployment with some sample data
response = ml_client.online_endpoints.invoke(
    endpoint_name=online_endpoint_name,
    deployment_name="blue",
    request_file="sample-data.json",
)

if response[1]=='1':
    print("Diabetic")
else:
    print ("Not diabetic")

Optionally, you can change the values in the `sample-data.json` file to try and get a different prediction.

## List endpoints

Although you can view all endpoints in the Studio, you can also list all endpoints using the SDK:

In [ ]:
endpoints = ml_client.online_endpoints.list()
for endp in endpoints:
    print(endp.name)

## Get endpoint details

If you want more information about a specific endpoint, you can explore the details using the SDK too.

In [ ]:
# Get the details for online endpoint
endpoint = ml_client.online_endpoints.get(name=online_endpoint_name)

# existing traffic details
print(endpoint.traffic)

# Get the scoring URI
print(endpoint.scoring_uri)

## Delete the endpoint and deployment

As an endpoint is always available, it can't be paused to save costs. To avoid unnecessary costs, delete the endpoint.

In [ ]:
ml_client.online_endpoints.begin_delete(name=online_endpoint_name)